In [1]:
import os

In [2]:
%pwd

'd:\\Programming\\Projects\\Plant-Disease-Classification-using-MLflow-and-DVC\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Programming\\Projects\\Plant-Disease-Classification-using-MLflow-and-DVC'

In [5]:
# Collect MLflow URI
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/rakukanteki/Plant-Disease-Classification-using-MLflow-and-DVC.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]= "rakukanteki"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "36c96502507cc4f5a19de049d18773a70292da82"

In [6]:
import tensorflow as tf

In [7]:
# Load Model.
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [8]:
from dataclasses import dataclass
from pathlib import Path

# Define Entity.
@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [9]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [ ]:
# Configuration Manager.
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/PLD_3_Classes_256",
            mlflow_uri="https://dagshub.com/rakukanteki/Plant-Disease-Classification-using-MLflow-and-DVC.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )

        return eval_config

In [11]:
# Create Components.
import tensorflow as tf
import urllib.request as request
from zipfile import ZipFile
import time
import numpy as np
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [12]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [13]:
# Pipeline
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2025-03-23 16:28:41,937: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-03-23 16:28:41,952: INFO: common: yaml file: params.yaml loaded successfully]
[2025-03-23 16:28:41,954: INFO: common: created directory at: artifacts]
Found 1220 images belonging to 3 classes.
42/77 [===============>..............] - ETA: 1:24 - loss: 16.9265 - accuracy: 0.8199Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\radwa\miniconda3\envs\plant\lib\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\radwa\AppData\Local\Temp\ipykernel_8424\3771632981.py", line 6, in <module>
    evaluation.evaluation()
  File "C:\Users\radwa\AppData\Local\Temp\ipykernel_8424\3392365584.py", line 39, in evaluation
    self.score = model.evaluate(self.valid_generator)
  File "c:\Users\radwa\miniconda3\envs\plant\lib\site-packages\keras\utils\traceback_utils.py", line 65, in error_handler
    return fn(*args, **kwargs)
  File "c:\Users\radwa\miniconda3\envs\plant\lib\site-packages\keras\engine\training.py", line 2072, in evaluate
    tmp_logs = self.test_function(iterator)
  File "c:\Users\radwa\miniconda3\envs\plant\lib\site-packages\tensorflow\python\util\traceback_utils.py", line 150, in error_handler
    return fn(*args, **kwargs)
  File "c:\Users\radwa\miniconda3\envs\plant